In [ ]:
from yugiquery import *
header('Bandai')

---

# Data aquisition

## Fetch online data

In [ ]:
timestamp = pd.Timestamp.now()

In [ ]:
bandai_df = fetch_bandai()

# Check changes

## Load previous data

In [ ]:
# Get list of files
files_list = sorted(glob.glob('../data/all_bandai_*.csv'), key=os.path.getctime, reverse=True)
# Get second newest file if exist
if files_list:
    # Load csv avoiding converting "NA" to NaN
    previous_df = pd.read_csv(files_list[0], dtype=object, keep_default_na=False, na_values=[''])
    # Force dtypes to match current df
    previous_df = previous_df.astype(bandai_df[previous_df.columns.intersection(bandai_df.columns)].dtypes.to_dict())
    previous_ts = pd.to_datetime(os.path.basename(files_list[0]).split('_')[-1].rstrip('.csv'))
    print('File loaded')
else:
    previous_df = None
    print('No older files')

## Generate changelog

In [ ]:
if previous_df is None:
    changelog=None
    print('Skipped')
else:
    changelog = generate_changelog(previous_df, bandai_df, col = 'Name')
    if not changelog.empty:
        display(changelog)
        changelog.to_csv(f'../data/bandai_changelog_{timestamp.isoformat(timespec="minutes")}_{previous_ts.isoformat(timespec="minutes")}.csv', index = True)
        print('Changelog saved')

## Save data

In [ ]:
if changelog is not None and changelog.empty:
    print('No changes. New data not saved')
else:
    bandai_df.to_csv(f'../data/all_bandai_{timestamp.isoformat(timespec="minutes")}.csv', index=False)
    print('Data saved')

# Data visualization

In [ ]:
bandai_df

# Epilogue

In [ ]:
benchmark('bandai',timestamp)

In [ ]:
footer()

## HTML export

In [ ]:
# Save notebook on disck before generating HTML report
save_notebook()

In [ ]:
! jupyter nbconvert Bandai.ipynb --output-dir='../' --to=HTML --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags='exclude' --TemplateExporter.exclude_input=True --TemplateExporter.exclude_input_prompt=True --TemplateExporter.exclude_output_prompt=True

## Git

In [ ]:
! git add "../*[Bb]andai*"

In [ ]:
! git commit -m {"'Bandai update-" + timestamp.isoformat() + "'"}